In [2]:
import pandas as pd  # Импорт библиотеки для работы с данными в формате таблицы (DataFrame)
import numpy as np  # Импорт библиотеки для работы с числовыми данными
import seaborn as sns  # Импорт библиотеки для визуализации данных
import matplotlib.pyplot as plt  # Импорт библиотеки для построения графиков
import matplotlib.mlab as mlab  # Импорт библиотеки для работы с графиками
import matplotlib  # Импорт основной библиотеки для работы с графиками
plt.style.use('ggplot')  # Задание стиля графиков
%matplotlib inline  # Встроенная магическая команда для отображения графиков в Jupyter Notebook
matplotlib.rcParams['figure.figsize'] = (10,10)  # Задание размеров графиков
pd.options.mode.chained_assignment = None  # Отключение предупреждений о цепочечных присваиваниях
from sklearn.model_selection import train_test_split as tts  # Импорт функции для разделения данных на обучающую и тестовую выборки
from sklearn.ensemble import ExtraTreesClassifier  # Импорт класса для построения модели на основе деревьев
from sklearn.ensemble import GradientBoostingRegressor  # Импорт класса для построения градиентного бустинга
from sklearn.metrics import mean_absolute_error as mae, r2_score, mean_squared_error as mse  # Импорт метрик для оценки качества модели
from sklearn.pipeline import Pipeline  # Импорт класса для создания пайплайна обработки данных
from sklearn.compose import ColumnTransformer  # Импорт класса для преобразования данных по столбцам
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler  # Импорт классов для кодирования категориальных признаков и масштабирования числовых
from sklearn.impute import SimpleImputer  # Импорт класса для заполнения пропущенных значений в данных

# Скачиваем, записываем и просматриваем данные файлов.
train = pd.read_csv('train.csv')
display(train)

macro = pd.read_csv('macro.csv')
display(macro)
macro['timestamp'].describe()

# Объединяем датафреймы по времени.
macro['timestamp'] = pd.to_datetime(macro['timestamp'])
train['timestamp'] = pd.to_datetime(train['timestamp'])
df = pd.merge(train, macro, on='timestamp')
display(df)

# Поиск категориальных признаков
i = 0
for column in df.select_dtypes(include='object'):
    i = i + 1
    print(i)
    print(column)
    print('Uniq values:', df[column].nunique())
    print(df[column].unique())

# Присваивание числовых значений категориальным столбцам.
pt = {'Investment': 1, 'OwnerOccupier': 0}
df['product_type'] = df['product_type'].map(pt)

ec = {'poor': 1, 'satisfactory': 2, 'good': 3, 'excellent': 4, 'no data': np.nan}
df['ecology'] = df['ecology'].map(ec)

df['child_on_acc_pre_school'] = df['child_on_acc_pre_school'].replace('#!', np.nan)

cmo = ['child_on_acc_pre_school', 'modern_education_share', 'old_education_build_share']

# Замена запятых на точки и преобразование строк в числа с плавающей запятой.
for column in cmo:
    df[column] = df[column].str.replace(',', '.').astype(float)

# Преобразование бинарных столбцов в числовой формат.
bin_cols = ['culture_objects_top_25',
            'thermal_power_plant_raion',
            'incineration_raion',
            'oil_chemistry_raion',
            'radiation_raion',
            'railroad_terminal_raion',
            'big_market_raion',
            'nuclear_reactor_raion',
            'detention_facility_raion',
            'water_1line',
            'big_road1_1line',
            'railroad_1line']
df[bin_cols] = df[bin_cols].applymap(lambda x: 1 if x == 'yes' else 0)

# Создание нового столбца с объединением редких значений.
suba = (df['sub_area'].value_counts() >= 100).loc[lambda x: x == True].index.tolist()
df['sub_area_new'] = np.where(df['sub_area'].isin(suba), df['sub_area'], 'Other')
df['sub_area_new'].value_counts()

# Вывод информации до и после объединения значений в столбце 'sub_area'.
print('До объединения -\n', df['sub_area'].value_counts())
print('\nПосле объединения -\n', df['sub_area_new'].value_counts())

# Удаление столбца 'sub_area'.
df = df.drop('sub_area', axis=1)

# Поиск аномалий и замена значений в столбце 'state'.
print(df['state'].value_counts())
df['state'].replace(33, 3, inplace=True)
print(df['state'].value_counts())

# Замена значений в столбце 'build_year'.
print(df['build_year'].value_counts())
bld_yr_replace = {20052009: 2009, 0: np.nan, 1: np.nan, 2: np.nan, 3: np.nan, 20: 2020, 215: 2015, 4965: 1965, 71: 1971}
df['build_year'].replace(bld_yr_replace, inplace=True)
print(df['build_year'].value_counts())

# Замена значений в столбце 'max_floor'.
print(df['max_floor'].unique())
max_flr_replace = {117: 17, 99: np.nan, 0: np.nan}
df['max_floor'].replace(max_flr_replace, inplace=True)
print(df['max_floor'].unique

# Замена значений в столбце 'full_sq'.
full_sq_replace = {5326: np.nan, 0: np.nan}
df['full_sq'].replace(full_sq_replace, inplace=True)
print(df['full_sq'].unique())

# Удаление неинформативных признаков.
df = df.drop(['id',
              'ID_big_road1',
              'ID_big_road2',
              'ID_bus_terminal',
              'ID_metro',
              'ID_railroad_station_avto',
              'ID_railroad_station_walk',
              'ID_railroad_terminal'], axis=1)

# Поиск пропусков цены и дупликатов, удаление.
print('Количество пропусков цены -', df['price_doc'].isna().sum())
print('Количество дупликатов -', df.duplicated().sum())
df = df.drop_duplicates()
print('Теперь дупликатов -', df.duplicated().sum())

# Удаляем признаки, которые заполнены менее чем на 55 процентов (уровень пропусков выше 45%).
i = 0
for col in df.columns:
    pct_missing = np.mean(df[col].isnull())
    if pct_missing * 100 > 45:
        df.drop([str(col)], axis=1, inplace=True)

# Вывод процента пропущенных значений для каждого признака.
for col in df.columns:
    pct_missing = np.mean(df[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing * 100)))
    i = i + 1
    print(i)

# Создаем индикатор для строк с пропущенными данными.
for col in df.columns:
    missing = df[col].isnull()
    num_missing = np.sum(missing)

    if num_missing > 0:
        print('created missing indicator for: {}'.format(col))
        df['{}_ismissing'.format(col)] = missing

# Формируем список колонок с индикаторами пропущенных значений.
ismissing_cols = [col for col in df.columns if 'ismissing' in col]
df['num_missing'] = df[ismissing_cols].sum(axis=1)
display(df)

# Удаляем строки с большим количеством пропусков (больше 35).
ind_missing = df[df['num_missing'] > 35].index
df.drop(ind_missing, axis=0, inplace=True)
display(df)

df_new = df.select_dtypes(include=[np.number]): Выбирает только числовые столбцы из исходного DataFrame df и сохраняет результат в новый DataFrame df_new.

df_new = df_new.dropna(): Удаляет строки с пропущенными значениями из нового DataFrame df_new.

df_new['price_doc_right'] = df_new['price_doc']: #Создает новый столбец 'price_doc_right', который является копией столбца 'price_doc'.

df_new.drop('price_doc', axis=1, inplace=True): #Удаляет столбец 'price_doc' из DataFrame df_new.

array = df_new.values: #Преобразует DataFrame df_new в массив NumPy.

X = array[:,0:373]: Выделяет все строки и первые 373 столбца массива array и сохраняет их в переменной X. Это матрица признаков.

Y = array[:,373]: Выделяет все строки и последний столбец массива array и сохраняет его в переменной Y. Это вектор целевой переменной.

model = ExtraTreesClassifier(): Создает экземпляр класса ExtraTreesClassifier, который представляет собой модель на основе деревьев решений для задачи классификации.

model.fit(X, Y): Обучает модель на матрице признаков X и векторе целевой переменной Y.

print(model.feature_importances_): Выводит важности признаков, рассчитанные моделью. Каждый элемент этого массива представляет собой важность соответствующего признака.

Таким образом, код использует алгоритм Extra Trees (экстремальные деревья) для определения важности признаков в данных и выводит результаты на экран. Важности признаков представлены в виде массива чисел, где каждое число отражает вклад соответствующего признака в обучение модели.

# 10 самых влияющих на цену недвижимости признаков.
tree_df = pd.DataFrame(data=model.feature_importances_, index=[list(range(1, 374))])
tree_df.sort_values(0, ascending=False).head(10)

# Корреляция цены на недвижимость в зависимости от жилплощади.
agg = df_new.groupby('full_sq')['price_doc_right'].mean()
agg.plot()

# Корреляция цены на недвижимость в зависимости от даты.
agg2 = df.groupby('timestamp')['price_doc'].mean()
agg2.plot()

# Какая цена на недвижимость встречается чаще.
df['price_doc'].hist(bins=100)

# Разделение датасета на обучающую и тестовую выборку с помощью train_test_split.
X_new = df.copy()
X_new = X_new[X_new.select_dtypes(include='number').columns]
X_new = X_new.dropna()
Y_new = np.array(X_new['price_doc'])
X_new = X_new.drop('price_doc', axis=1)

X_train0, X_test0, Y_train0, Y_test0 = tts(X_new, Y_new, test_size=0.2, shuffle=True)

# Обучение регрессионной модели с помощью GradientBoostingRegressor.
regress = GradientBoostingRegressor()
regress.fit(X_train0, Y_train0)

Y_train_pred0 = regress.predict(X_train0)
Y_test_pred0 = regress.predict(X_test0)

# Вывод метрик качества модели.
print('Train MAE', mae(Y_train0, Y_train_pred0), ' ', 'Test MAE:', mae(Y_test0, Y_test_pred0))
print('Train MSE', mse(Y_train0, Y_train_pred0), ' ', 'Test MSE:', mse(Y_test0, Y_test_pred0))
print('Train R2', r2_score(Y_train0, Y_train_pred0), ' ', 'Test R2:', r2_score(Y_test0, Y_test_pred0))

# Создаем новые фичи по признаку timestamp.
df['month'] = (df['timestamp'].dt.to_period('M'))
df['dow'] = df['timestamp'].dt.dayofweek
df['dom'] = df['timestamp'].dt.day
df['doy'] = df['timestamp'].dt.dayofyear
df['month'] = df['timestamp'].dt.month
df = df.drop('timestamp', axis=1)

# Разделение датасета на обучающую и тестовую выборку с помощью train_test_split.
X = df.copy()
Y = X.pop('price_doc')
X_train, X_test, Y_train, Y_test = tts(X, Y, test_size=0.2, stratify=X['sub_area_new'], shuffle=True)

# Объявление и подготовка к реализации новых фич.
features1 = ['sub_area_new']
features2 = ['thermal_power_plant_raion', 'incineration_raion', 'oil_chemistry_raion', 'radiation_raion',
             'railroad_terminal_raion', 'big_market_raion', 'nuclear_reactor_raion', 'detention_facility_raion',
             'water_1line', 'big_road1_1line', 'railroad_1line', 'railroad_1line', 'product_type', 'ecology',
             'material', 'state', 'month', 'dow', 'dom', 'doy', 'month']

features = features1 + features2

list_features =list(set(X_train.columns.tolist()) - set(features))

# Подготовка к реализации новых фич.
X_train_trans = X_train.copy()
X_test_trans = X_test.copy()

# Преобразование категориальных признаков.
categorial_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                                         ('onehot', OneHotEncoder(handle_unknown='ignore', sparse=False))])

# Заполнение пропущенных значений для числовых признаков и их масштабирование.
imputer_scaler_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                                             ('scaler', MinMaxScaler())])

# Применение преобразований к различным группам признаков.
preprocessor = ColumnTransformer(transformers=[('categorical', categorial_transformer, features1),
                                               ('imputer_scaler', imputer_scaler_transformer, features2),
                                               ('passthrough', 'passthrough', list_features)])

# Построение пайплайна с преобразованием данных и обучением модели.
reg = Pipeline(steps=[('preprocessor', preprocessor),
                      ('regressor', GradientBoostingRegressor())])

# Обучение модели на обучающей выборке.
reg.fit(X_train_trans, Y_train)

# Прогнозы для обучающей и тестовой выборок.
Y_train_pred = reg.predict(X_train_trans)
Y_test_pred = reg.predict(X_test_trans)

# Вывод метрик качества новой модели.
print('Train MAE:', mae(Y_train, Y_train_pred), 'Test MAE:', mae(Y_test, Y_test_pred))
print('Train MSE:', mse(Y_train, Y_train_pred), 'Test MSE:', mse(Y_test, Y_test_pred))
print('Train R2:', r2_score(Y_train, Y_train_pred), 'Test R2:', r2_score(Y_test, Y_test_pred))

SyntaxError: '(' was never closed (2793691028.py, line 100)